# Modelo TEST

## Carga de datos y librerias

In [1]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
import xgboost as xgb
import pickle
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ParameterGrid

In [2]:
pathfile1 = '../data/Loan_test_set.csv'

In [9]:
loans = pd.read_csv(pathfile1, header = 1, skipfooter = 2, engine = 'python')

## Limpieza datos

In [4]:
loans = loans.duplicated(keep = False)

In [5]:
duplicated = loans.duplicated()
duplicated_true = duplicated[duplicated == 'True']
duplicated_true.count()

0

In [10]:
loans_descarte_1 = loans.loc[:, ['funded_amnt', 'funded_amnt_inv', 'last_fico_range_high', 'member_id', 'last_fico_range_low', 'id', 'issue_d', 'last_credit_pull_d', 'last_pymnt_amnt', 'last_pymnt_d', 'loan_amnt', 'next_pymnt_d', 'out_prncp', 'out_prncp_inv', 'pymnt_plan', 'recoveries', 'sub_grade', 'total_pymnt', 'total_pymnt_inv', 'total_rec_int', 'total_rec_late_fee', 'total_rec_prncp', 'url', 'hardship_type', 'deferral_term', 'hardship_amount', 'hardship_start_date', 'hardship_end_date', 'payment_plan_start_date', 'hardship_length', 'hardship_dpd', 'hardship_loan_status', 'orig_projected_additional_accrued_interest', 'hardship_payoff_balance_amount', 'hardship_last_payment_amount', 'disbursement_method', 'debt_settlement_flag', 'debt_settlement_flag_date', 'settlement_status', 'settlement_date', 'settlement_amount', 'settlement_percentage', 'settlement_term', 'hardship_flag', 'hardship_reason', 'hardship_status', 'collection_recovery_fee']]
list(loans_descarte_1)

['funded_amnt',
 'funded_amnt_inv',
 'last_fico_range_high',
 'member_id',
 'last_fico_range_low',
 'id',
 'issue_d',
 'last_credit_pull_d',
 'last_pymnt_amnt',
 'last_pymnt_d',
 'loan_amnt',
 'next_pymnt_d',
 'out_prncp',
 'out_prncp_inv',
 'pymnt_plan',
 'recoveries',
 'sub_grade',
 'total_pymnt',
 'total_pymnt_inv',
 'total_rec_int',
 'total_rec_late_fee',
 'total_rec_prncp',
 'url',
 'hardship_type',
 'deferral_term',
 'hardship_amount',
 'hardship_start_date',
 'hardship_end_date',
 'payment_plan_start_date',
 'hardship_length',
 'hardship_dpd',
 'hardship_loan_status',
 'orig_projected_additional_accrued_interest',
 'hardship_payoff_balance_amount',
 'hardship_last_payment_amount',
 'disbursement_method',
 'debt_settlement_flag',
 'debt_settlement_flag_date',
 'settlement_status',
 'settlement_date',
 'settlement_amount',
 'settlement_percentage',
 'settlement_term',
 'hardship_flag',
 'hardship_reason',
 'hardship_status',
 'collection_recovery_fee']

In [11]:
loans_1 = loans.drop(loans_descarte_1, axis = 1)
list(loans_1)

['term',
 'int_rate',
 'installment',
 'grade',
 'emp_title',
 'emp_length',
 'home_ownership',
 'annual_inc',
 'verification_status',
 'loan_status',
 'desc',
 'purpose',
 'title',
 'zip_code',
 'addr_state',
 'dti',
 'delinq_2yrs',
 'earliest_cr_line',
 'fico_range_low',
 'fico_range_high',
 'inq_last_6mths',
 'mths_since_last_delinq',
 'mths_since_last_record',
 'open_acc',
 'pub_rec',
 'revol_bal',
 'revol_util',
 'total_acc',
 'initial_list_status',
 'collections_12_mths_ex_med',
 'mths_since_last_major_derog',
 'policy_code',
 'application_type',
 'annual_inc_joint',
 'dti_joint',
 'verification_status_joint',
 'acc_now_delinq',
 'tot_coll_amt',
 'tot_cur_bal',
 'open_acc_6m',
 'open_act_il',
 'open_il_12m',
 'open_il_24m',
 'mths_since_rcnt_il',
 'total_bal_il',
 'il_util',
 'open_rv_12m',
 'open_rv_24m',
 'max_bal_bc',
 'all_util',
 'total_rev_hi_lim',
 'inq_fi',
 'total_cu_tl',
 'inq_last_12m',
 'acc_open_past_24mths',
 'avg_cur_bal',
 'bc_open_to_buy',
 'bc_util',
 'chargeoff

In [12]:
loans_descarte_relevancia = loans_1.loc[:, ['int_rate', 'emp_title', 'zip_code']]
loans_1 = loans_1.drop(loans_descarte_relevancia, axis = 1)

In [13]:
loans_1['fico_avg'] = (loans_1['fico_range_high'] + loans_1['fico_range_low']) / 2

In [14]:
descarte_fico = loans_1.loc[:, ['fico_range_high', 'fico_range_low']]
loans_1 = loans_1.drop(descarte_fico, axis = 1)

In [15]:
loans_1 = loans_1[loans_1.loan_status != 'Current']
loans_1[loans_1.loan_status == 'Current']

,term,installment,grade,emp_length,home_ownership,annual_inc,verification_status,loan_status,desc,purpose,...,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,fico_avg


In [16]:
loans_1['loan_status'].unique()

array(['Fully Paid', 'Late (16-30 days)', 'Late (31-120 days)',
       'Charged Off', 'In Grace Period', 'Default'], dtype=object)

In [17]:
loan_status_dummy = map(lambda x: 1 if x == 'Fully Paid' else 0, loans_1.loan_status)
loans_1['loan_status'] = list(loan_status_dummy)
loans_1['loan_status'] = loans_1['loan_status'].astype(float)

In [18]:
loans_1['loan_status'].unique()

array([1., 0.])

# División en dos datasets

In [19]:
loans_joint = loans_1[loans_1.application_type == 'Joint App']
loans_ind = loans_1[loans_1.application_type == 'Individual']

# Individual

In [20]:
loans_descarte_joint = loans_1.loc[:, ['dti_joint', 'annual_inc_joint', 'verification_status_joint', 'sec_app_mths_since_last_major_derog', 'sec_app_revol_util', 'sec_app_inq_last_6mths', 'sec_app_collections_12_mths_ex_med', 'revol_bal_joint', 'sec_app_fico_range_low', 'sec_app_earliest_cr_line', 'sec_app_fico_range_high', 'sec_app_mort_acc', 'sec_app_open_acc', 'sec_app_open_act_il', 'sec_app_num_rev_accts', 'sec_app_chargeoff_within_12_mths']]
list(loans_descarte_joint)

['dti_joint',
 'annual_inc_joint',
 'verification_status_joint',
 'sec_app_mths_since_last_major_derog',
 'sec_app_revol_util',
 'sec_app_inq_last_6mths',
 'sec_app_collections_12_mths_ex_med',
 'revol_bal_joint',
 'sec_app_fico_range_low',
 'sec_app_earliest_cr_line',
 'sec_app_fico_range_high',
 'sec_app_mort_acc',
 'sec_app_open_acc',
 'sec_app_open_act_il',
 'sec_app_num_rev_accts',
 'sec_app_chargeoff_within_12_mths']

In [21]:
loans_ind = loans_ind.drop(loans_descarte_joint, axis = 1)
list(loans_ind)

['term',
 'installment',
 'grade',
 'emp_length',
 'home_ownership',
 'annual_inc',
 'verification_status',
 'loan_status',
 'desc',
 'purpose',
 'title',
 'addr_state',
 'dti',
 'delinq_2yrs',
 'earliest_cr_line',
 'inq_last_6mths',
 'mths_since_last_delinq',
 'mths_since_last_record',
 'open_acc',
 'pub_rec',
 'revol_bal',
 'revol_util',
 'total_acc',
 'initial_list_status',
 'collections_12_mths_ex_med',
 'mths_since_last_major_derog',
 'policy_code',
 'application_type',
 'acc_now_delinq',
 'tot_coll_amt',
 'tot_cur_bal',
 'open_acc_6m',
 'open_act_il',
 'open_il_12m',
 'open_il_24m',
 'mths_since_rcnt_il',
 'total_bal_il',
 'il_util',
 'open_rv_12m',
 'open_rv_24m',
 'max_bal_bc',
 'all_util',
 'total_rev_hi_lim',
 'inq_fi',
 'total_cu_tl',
 'inq_last_12m',
 'acc_open_past_24mths',
 'avg_cur_bal',
 'bc_open_to_buy',
 'bc_util',
 'chargeoff_within_12_mths',
 'delinq_amnt',
 'mo_sin_old_il_acct',
 'mo_sin_old_rev_tl_op',
 'mo_sin_rcnt_rev_tl_op',
 'mo_sin_rcnt_tl',
 'mort_acc',
 'mt

In [22]:
loans_descarte_2 = loans_1.loc[:, ['il_util', 'mths_since_rcnt_il', 'all_util', 'open_rv_24m', 'open_acc_6m', 'open_act_il', 'open_il_12m', 'open_il_24m', 'total_bal_il', 'max_bal_bc', 'inq_fi', 'total_cu_tl', 'inq_last_12m', 'open_rv_12m', 'mths_since_recent_bc_dlq', 'mths_since_recent_revol_delinq', 'desc', 'mths_since_last_record', 'mths_since_last_major_derog']]
list(loans_descarte_2)

['il_util',
 'mths_since_rcnt_il',
 'all_util',
 'open_rv_24m',
 'open_acc_6m',
 'open_act_il',
 'open_il_12m',
 'open_il_24m',
 'total_bal_il',
 'max_bal_bc',
 'inq_fi',
 'total_cu_tl',
 'inq_last_12m',
 'open_rv_12m',
 'mths_since_recent_bc_dlq',
 'mths_since_recent_revol_delinq',
 'desc',
 'mths_since_last_record',
 'mths_since_last_major_derog']

In [23]:
loans_ind = loans_ind.drop(loans_descarte_2, axis = 1)
list(loans_ind)

['term',
 'installment',
 'grade',
 'emp_length',
 'home_ownership',
 'annual_inc',
 'verification_status',
 'loan_status',
 'purpose',
 'title',
 'addr_state',
 'dti',
 'delinq_2yrs',
 'earliest_cr_line',
 'inq_last_6mths',
 'mths_since_last_delinq',
 'open_acc',
 'pub_rec',
 'revol_bal',
 'revol_util',
 'total_acc',
 'initial_list_status',
 'collections_12_mths_ex_med',
 'policy_code',
 'application_type',
 'acc_now_delinq',
 'tot_coll_amt',
 'tot_cur_bal',
 'total_rev_hi_lim',
 'acc_open_past_24mths',
 'avg_cur_bal',
 'bc_open_to_buy',
 'bc_util',
 'chargeoff_within_12_mths',
 'delinq_amnt',
 'mo_sin_old_il_acct',
 'mo_sin_old_rev_tl_op',
 'mo_sin_rcnt_rev_tl_op',
 'mo_sin_rcnt_tl',
 'mort_acc',
 'mths_since_recent_bc',
 'mths_since_recent_inq',
 'num_accts_ever_120_pd',
 'num_actv_bc_tl',
 'num_actv_rev_tl',
 'num_bc_sats',
 'num_bc_tl',
 'num_il_tl',
 'num_op_rev_tl',
 'num_rev_accts',
 'num_rev_tl_bal_gt_0',
 'num_sats',
 'num_tl_120dpd_2m',
 'num_tl_30dpd',
 'num_tl_90g_dpd_24m'

In [24]:
split_term = map(lambda x: x.split(' ')[1], loans_ind.term)
loans_ind.term = list(split_term)
loans_ind['term'] = loans_ind['term'].astype(float)

In [25]:
loans_ind['revol_util'] = loans_ind.revol_util.str.strip('%').astype(float)
loans_ind['earliest_cr_line'] = pd.to_datetime(loans_ind['earliest_cr_line'])

In [26]:
loans_ind['emp_length'] = loans_ind.emp_length.str.strip('<')
loans_ind['emp_length'] = loans_ind.emp_length.str.strip('+ years')
loans_ind['emp_length'] = loans_ind['emp_length'].astype('float')

In [27]:
def delinq_range(x):
    if x <= 12:
        return 10
    elif x <= 24:
        return 9
    elif x <= 36:
        return 8
    elif x <= 48:
        return 7
    elif x <= 60:
        return 6
    elif x <= 72:
        return 5
    elif x <= 84:
        return 4
    elif x <= 96:
        return 3
    elif x <= 108:
        return 2
    elif x > 108:
        return 1
    else:
        return 0

In [28]:
loans_ind['mths_since_last_delinq'] = loans_ind.mths_since_last_delinq.map(delinq_range)

loans_ind['mths_since_last_delinq']

16        0
22       10
26        0
38        8
40        5
         ..
96746    10
96750     9
96761     0
96763     0
96772     0
Name: mths_since_last_delinq, Length: 17827, dtype: int64

In [29]:
descarte_valores_unicos = ['delinq_2yrs', 'pub_rec', 'collections_12_mths_ex_med', 'policy_code', 'acc_now_delinq', 'tot_coll_amt', 'chargeoff_within_12_mths', 'delinq_amnt', 'num_accts_ever_120_pd', 'num_tl_120dpd_2m', 'num_tl_30dpd', 'num_tl_90g_dpd_24m', 'tax_liens']
loans_ind = loans_ind.drop(descarte_valores_unicos, axis = 1)

In [30]:
descarte_cor = ['num_bc_tl', 'num_bc_sats', 'num_rev_tl_bal_gt_0', 'num_op_rev_tl', 'num_bc_sats']
loans_ind = loans_ind.drop(descarte_cor, axis = 1)

In [31]:
loans_ind['loan_status'].value_counts()

1.0    12651
0.0     5176
Name: loan_status, dtype: int64

# Joint

In [32]:
loans_descartes_joint_na = loans_joint.loc[:,['sec_app_mths_since_last_major_derog', 'mths_since_recent_bc_dlq', 'mths_since_recent_revol_delinq', 'desc', 'mths_since_last_record', 'mths_since_last_record', 'mths_since_last_major_derog']]

In [33]:
loans_joint = loans_joint.drop(loans_descartes_joint_na, axis = 1)
loans_joint

,term,installment,grade,emp_length,home_ownership,annual_inc,verification_status,loan_status,purpose,title,...,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,fico_avg
132,60 months,437.63,C,NaN,MORTGAGE,34000.0,Verified,1.0,debt_consolidation,Debt consolidation,...,Jan-1999,0.0,2.0,10.0,80.3,1.0,17.0,0.0,0.0,662.0
154,36 months,854.32,C,10+ years,MORTGAGE,90000.0,Verified,1.0,home_improvement,Home improvement,...,Jun-2002,1.0,0.0,9.0,50.7,1.0,13.0,0.0,0.0,677.0
212,60 months,812.66,C,5 years,MORTGAGE,31872.0,Verified,1.0,debt_consolidation,Debt consolidation,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,727.0
244,36 months,292.17,F,1 year,RENT,35000.0,Verified,1.0,debt_consolidation,Debt consolidation,...,Sep-1999,4.0,0.0,10.0,49.2,2.0,10.0,0.0,2.0,662.0
400,60 months,805.17,C,3 years,MORTGAGE,98000.0,Verified,0.0,debt_consolidation,Debt consolidation,...,Sep-2010,1.0,0.0,25.0,60.6,2.0,22.0,0.0,0.0,702.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96192,36 months,503.16,B,10+ years,RENT,34000.0,Source Verified,1.0,debt_consolidation,Debt consolidation,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,787.0
96251,60 months,768.99,F,NaN,OWN,25000.0,Verified,0.0,debt_consolidation,Debt consolidation,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,692.0
96614,60 months,815.66,B,1 year,RENT,150000.0,Verified,1.0,debt_consolidation,Debt consolidation,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,742.0
96630,60 months,393.72,F,NaN,OWN,49500.0,Not Verified,0.0,house,Home buying,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,672.0


In [34]:
split_term = map(lambda x: x.split(' ')[1], loans_joint.term)
loans_joint.term = list(split_term)
loans_joint['term'] = loans_joint['term'].astype(float)

In [35]:
loans_joint['revol_util'] = loans_joint.revol_util.str.strip('%').astype(float)
loans_joint['earliest_cr_line'] = pd.to_datetime(loans_joint['earliest_cr_line'])

In [36]:
loans_joint['emp_length'] = loans_joint.emp_length.str.strip('<')
loans_joint['emp_length'] = loans_joint.emp_length.str.strip('+ years')
loans_joint['emp_length'] = loans_joint['emp_length'].astype('float')

In [37]:
def delinq_range(x):
    if x <= 12:
        return 10
    elif x <= 24:
        return 9
    elif x <= 36:
        return 8
    elif x <= 48:
        return 7
    elif x <= 60:
        return 6
    elif x <= 72:
        return 5
    elif x <= 84:
        return 4
    elif x <= 96:
        return 3
    elif x <= 108:
        return 2
    elif x > 108:
        return 1
    else:
        return 0

In [38]:
loans_joint['mths_since_last_delinq'] = loans_joint.mths_since_last_delinq.map(delinq_range)

loans_joint['mths_since_last_delinq']

132      0
154      9
212      8
244      0
400      0
        ..
96192    0
96251    0
96614    0
96630    0
96650    0
Name: mths_since_last_delinq, Length: 700, dtype: int64

In [39]:
loans_joint_descartes_2 = loans_joint.loc[:, ['collections_12_mths_ex_med', 'policy_code', 'acc_now_delinq', 'tot_coll_amt', 'num_tl_30dpd', 'num_tl_90g_dpd_24m']]

In [40]:
loans_joint = loans_joint.drop(loans_joint_descartes_2, axis = 1)
list(loans_joint)

['term',
 'installment',
 'grade',
 'emp_length',
 'home_ownership',
 'annual_inc',
 'verification_status',
 'loan_status',
 'purpose',
 'title',
 'addr_state',
 'dti',
 'delinq_2yrs',
 'earliest_cr_line',
 'inq_last_6mths',
 'mths_since_last_delinq',
 'open_acc',
 'pub_rec',
 'revol_bal',
 'revol_util',
 'total_acc',
 'initial_list_status',
 'application_type',
 'annual_inc_joint',
 'dti_joint',
 'verification_status_joint',
 'tot_cur_bal',
 'open_acc_6m',
 'open_act_il',
 'open_il_12m',
 'open_il_24m',
 'mths_since_rcnt_il',
 'total_bal_il',
 'il_util',
 'open_rv_12m',
 'open_rv_24m',
 'max_bal_bc',
 'all_util',
 'total_rev_hi_lim',
 'inq_fi',
 'total_cu_tl',
 'inq_last_12m',
 'acc_open_past_24mths',
 'avg_cur_bal',
 'bc_open_to_buy',
 'bc_util',
 'chargeoff_within_12_mths',
 'delinq_amnt',
 'mo_sin_old_il_acct',
 'mo_sin_old_rev_tl_op',
 'mo_sin_rcnt_rev_tl_op',
 'mo_sin_rcnt_tl',
 'mort_acc',
 'mths_since_recent_bc',
 'mths_since_recent_inq',
 'num_accts_ever_120_pd',
 'num_actv_bc

In [41]:
joint_descartes_3 = loans_joint.loc[:,['num_bc_sats', 'num_op_rev_tl', 'num_rev_tl_bal_gt_0']]

In [42]:
loans_joint = loans_joint.drop(joint_descartes_3, axis = 1)

# Test individual

# Carga del modelo

In [60]:
xgboost = pickle.load(open('model_xgboost.sav', 'rb'))

# Preparación del modelo

In [61]:
y = loans_ind['loan_status']
x = loans_ind.drop('loan_status', axis = 1)


In [62]:
numeric_transformer_0 = Pipeline(steps = [
    ('imputer_0', SimpleImputer(strategy = 'constant')),
    ('escalar', StandardScaler())
])

numeric_transformer_median = Pipeline(steps = [
    ('imputer_median', SimpleImputer(strategy = 'median')),
    ('escalar1', StandardScaler())
])

categoric_transformer  = Pipeline(steps = [
    ('imputer', SimpleImputer(strategy = 'constant', fill_value = 'sin_info')),
    ('onehot', OneHotEncoder(handle_unknown = 'ignore')) # mirar esto de handle_unknowns
])

In [63]:
numeric_features_0 = x.loc[:, ['bc_open_to_buy', 'mths_since_recent_bc', 'revol_util', 'emp_length', 'term', 'installment', 'annual_inc', 'inq_last_6mths',
                                      'mths_since_last_delinq', 'open_acc', 'revol_bal', 'total_acc', 'tot_cur_bal', 'acc_open_past_24mths',
                                       'mo_sin_old_il_acct', 'mo_sin_old_rev_tl_op',
                                      'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl', 'mort_acc', 'mths_since_recent_inq', 'num_actv_rev_tl',
                                       'num_sats', 'num_tl_op_past_12m', 'pct_tl_nvr_dlq', 'pub_rec_bankruptcies', 'tot_hi_cred_lim',
                                      'total_bal_ex_mort', 'total_bc_limit', 'total_il_high_credit_limit', 'fico_avg']].columns

numeric_features_median = x.loc[:, ['percent_bc_gt_75',
                                            'bc_util',
                                            'avg_cur_bal',
                                            'num_rev_accts',
                                            'dti']].columns


categoric_features = x.select_dtypes(include = ['object']).columns

In [64]:
preprocessor = ColumnTransformer(
        transformers = [
            ('num_0', numeric_transformer_0, numeric_features_0),
            ('num_median', numeric_transformer_median, numeric_features_median),
            ('cat', categoric_transformer, categoric_features)
        ]
)

# Ejecución del modelo

In [69]:
y.value_counts()

1.0    12651
0.0     5176
Name: loan_status, dtype: int64

In [80]:
xgboost.predict(x)

array([1., 1., 1., ..., 0., 1., 1.])

In [81]:
print("model score: %.3f" % xgboost.score(x, y))

model score: 0.715


# Test joint

# Carga del modelo

In [71]:
xgboost_joint = pickle.load(open('model_xgboost_joint.sav', 'rb'))

In [72]:
y_joint = loans_joint['loan_status']
x_joint = loans_joint.drop('loan_status', axis = 1)

# Preparación del modelo

In [73]:
numeric_transformer_0_joint = Pipeline(steps = [
    ('imputer_0_joint', SimpleImputer(strategy = 'constant')),
    ('escalar_joint', StandardScaler())
])


categoric_transformer_joint  = Pipeline(steps = [
    ('imputer_joint', SimpleImputer(strategy = 'constant', fill_value = 'sin_info')),
    ('onehot_joint', OneHotEncoder(handle_unknown = 'ignore')) # mirar esto de handle_unknowns
])

In [74]:
numeric_features_0_joint = x_joint.select_dtypes(include=['int64', 'float64']).columns

categoric_features_joint = x_joint.select_dtypes(include = ['object']).columns

In [75]:
preprocessor_joint = ColumnTransformer(
        transformers = [
            ('num_0_joint', numeric_transformer_0_joint, numeric_features_0_joint),
            ('cat_joint', categoric_transformer_joint, categoric_features_joint)
        ]
)

# Ejecución del modelo

In [82]:
xgboost_joint.predict(x)

ValueError: Number of features of the input must be equal to or greater than that of the fitted transformer. Transformer n_features is 85 and input n_features is 47.

In [79]:
print("model score: %.3f" % xgboost_joint.score(x, y))

ValueError: Number of features of the input must be equal to or greater than that of the fitted transformer. Transformer n_features is 85 and input n_features is 47.